In [2]:
%load_ext autoreload
%autoreload 2

!export TRITON_CACHE_DIR=/data/gpfs/projects/punim2662/.cache/triton
!export XDG_CONFIG_HOME=/data/gpfs/projects/punim2662/.config
!export VLLM_CACHE_DIR=/data/gpfs/projects/punim2662/.cache/vllm
!export VLLM_CACHE_ROOT=/data/gpfs/projects/punim2662/.cache/vllm
!export TORCH_HOME=/data/gpfs/projects/punim2662/.cache/torch/
!export TORCHINDUCTOR_CACHE_DIR=/data/gpfs/projects/punim2662/.cache/torch/inductor
!export CUDA_CACHE_PATH=/data/gpfs/projects/punim2662/.cache/nvidia/
!export HF_HOME=/data/gpfs/projects/punim2662/.cache/huggingface

# Hugging Face authentication token - set via environment variable before running
# Example: export HF_AUTH_TOKEN="your_token_here" in your shell before starting Jupyter
# Or set it in this notebook: import os; os.environ['HF_AUTH_TOKEN'] = "your_token_here"

### **Load the mmlu auxiliary split**


In [4]:
from routing_dataset.dataset_paths import *
import pickle
from datasets import load_dataset, get_dataset_config_names, Dataset
from routing_dataset.load_dataset import load_mmlu_pro_dataset
from routing_dataset.load_dataset import load_gsm8k_split
from routing_dataset.run_prompts import add_correct_labels
import pandas

In [ ]:
from routing_dataset.dataset_paths import (
    FINAL_HOTPOTQA_QWEN8B_TRAIN_FILE,
    FINAL_HOTPOTQA_QWEN8B_VAL_FILE,
    FINAL_HOTPOTQA_QWEN8B_TEST_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_TRAIN_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_VAL_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_TEST_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_TRAIN_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_VAL_FILE,
    FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_TEST_FILE,
    
)

from routing_dataset.load_dataset import stratfieid_train_validation_split

with open(FINAL_HOTPOTQA_QWEN8B_TRAIN_FILE, 'rb') as file:
    hotpotqa_train = pickle.load(file)

with open(FINAL_HOTPOTQA_QWEN8B_VAL_FILE, 'rb') as file:
    hotpotqa_val = pickle.load(file)

with open(FINAL_HOTPOTQA_QWEN8B_TEST_FILE, 'rb') as file:
    hotpotqa_test = pickle.load(file)

with open(FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_TRAIN_FILE, 'rb') as file:
    mmlu_all_pro_gsm8k_train = pickle.load(file)

with open(FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_VAL_FILE, 'rb') as file:
    mmlu_all_pro_gsm8k_val = pickle.load(file)

with open(FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_TEST_FILE, 'rb') as file:
    mmlu_all_pro_gsm8k_test = pickle.load(file)

hotpotqa_combined = pandas.concat([hotpotqa_train, hotpotqa_val, hotpotqa_test])
hotpotqa_combined['dataset_source'] = 'hotpotqa'

# Drop some columns
hotpotqa_combined = hotpotqa_combined.drop(columns=['id', 'context', 'question', 'answer', 'pred', ])

print(hotpotqa_combined.keys())

mmlu_all_pro_gsm8k_combined = pandas.concat([mmlu_all_pro_gsm8k_train, mmlu_all_pro_gsm8k_val, mmlu_all_pro_gsm8k_test])

mmlu_all_pro_gsm8k_combined = mmlu_all_pro_gsm8k_combined.drop(columns=['ground_truths', 'responses', 'extracted_answers'])

all_combined = pandas.concat([hotpotqa_combined, mmlu_all_pro_gsm8k_combined])

train, test, validation = stratfieid_train_validation_split(all_combined)

train.to_pickle(FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_TRAIN_FILE)
validation.to_pickle(FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_VAL_FILE)
test.to_pickle(FINAL_MMLU_ALL_PRO_GSM8K_QWEN8B_HOTPOTQA_QWEN8B_TEST_FILE)


Index(['prompts', 'correct_labels', 'dataset_source'], dtype='object')
234446
